In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
path = 'C:/Users/wjssm/Desktop/kaggle/Elo_Merchant_Category_Recommendation/data/'

train = pd.read_csv(path + 'train.csv') #key : card_id
test = pd.read_csv(path + 'test.csv') #key : card_id
history = pd.read_csv(path+'historical_transactions.csv') #key : card_id, merchant_id
new_history = pd.read_csv(path+'new_merchant_transactions.csv') #key : card_id, merchant_id
merchant = pd.read_csv(path+'merchants.csv') #key : merchant_id

In [5]:
train['year'] = pd.DatetimeIndex(train['first_active_month']).year
train['month'] = pd.DatetimeIndex(train['first_active_month']).month

test['year'] = pd.DatetimeIndex(test['first_active_month']).year
test['month'] = pd.DatetimeIndex(test['first_active_month']).month

In [6]:
#test data에서의 NA어떻게 할지...
test.loc[test.first_active_month.isnull(),:]

,first_active_month,card_id,feature_1,feature_2,feature_3,year,month
11578,NaN,C_ID_c27b4f80f7,5,2,1,NaN,NaN


In [7]:
#최빈값으로 채움
test.loc[test.card_id == 'C_ID_c27b4f80f7', 'year'] = 2017
test.loc[test.card_id == 'C_ID_c27b4f80f7', 'month'] = 12

In [8]:
train['year'] = train['year'].astype('category')
train['month'] = train['month'].astype('category')
test['year'] = test['year'].astype('category')
test['month'] = test['month'].astype('category')

### new_feature : transaction_count  카드 아이디별 거래 건수

In [9]:
#history card_id 별 거래수
trasaction_count = history.groupby(['card_id'])['authorized_flag'].count().reset_index(name = 'trasaction_count')
trasaction_count.head()

,card_id,trasaction_count
0,C_ID_00007093c1,149
1,C_ID_0001238066,123
2,C_ID_0001506ef0,66
3,C_ID_0001793786,216
4,C_ID_000183fdda,144


### history - installments outlier

In [27]:
history['installments'].value_counts()

 0      15411747
 1      11677522
 2        666416
 3        538207
 4        179525
-1        178159
 6        132634
 10       118827
 5        116090
 12        55064
 8         20474
 7         10906
 9          5772
 11          830
 999         188
Name: installments, dtype: int64

### 1. installments : one of a series of regular payments that you make until you have paid all the money you owe

In [28]:
history.loc[history['installments']==999, :]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
227606,N,C_ID_8b04d4734e,210,N,999,C,806,M_ID_0155450c84,-11,6.237415,2017-03-28 19:40:50,1.0,16,9
466436,N,C_ID_7670e569a0,333,N,999,C,813,M_ID_c9526f2dd3,-10,-0.443673,2017-04-12 21:06:28,5.0,21,29
759501,N,C_ID_652976a4bb,19,N,999,C,356,M_ID_2570628257,-8,0.469038,2017-06-05 09:22:18,1.0,9,21
1182532,N,C_ID_24fb1a2959,331,N,999,C,117,M_ID_67a62f2671,-3,0.208778,2017-11-05 01:29:18,1.0,16,2
1198327,N,C_ID_3742a35e45,333,N,999,C,307,M_ID_5f46ec7f98,-8,-0.521510,2017-05-01 11:17:28,5.0,21,19
1221217,N,C_ID_ba84d050f5,19,N,999,C,884,M_ID_8e7a785122,-8,-0.296127,2017-06-25 16:09:55,1.0,9,27
1368209,N,C_ID_5b6278eb25,69,N,999,C,683,M_ID_f86439cec0,-6,-0.674179,2017-08-23 12:39:48,1.0,9,34
1682455,N,C_ID_51ab074d19,69,N,999,C,412,M_ID_dfc0be4604,-5,-0.100768,2017-09-15 17:42:57,1.0,9,18
1767608,N,C_ID_9cea123f42,181,N,999,C,90,M_ID_596749cc5a,0,-0.318652,2018-02-23 17:29:00,5.0,5,22
2063505,N,C_ID_0a3724504a,279,N,999,C,667,M_ID_45bfdc72e8,-4,-0.259147,2017-10-13 18:46:21,1.0,9,21


In [38]:
#category가 B이면 installments 가 다 1인가?
history.loc[history.card_id=='C_ID_8b04d4734e', :].sort_values(['merchant_id','purchase_date']).loc[history.category_3=='B','installments'].value_counts()

1    482
Name: installments, dtype: int64

In [39]:
#category가 B이면 installments 가 다 1이다!
history.loc[history.category_3=='B','installments'].value_counts()

1    11677522
Name: installments, dtype: int64

In [43]:
history.loc[history.card_id=='C_ID_8b04d4734e', :].sort_values(['merchant_id','purchase_date'])

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
227364,Y,C_ID_8b04d4734e,69,N,1,B,511,M_ID_00a6ca8a8a,-11,-0.356218,2017-03-22 14:04:30,1.0,9,7
227330,Y,C_ID_8b04d4734e,69,N,1,B,511,M_ID_00a6ca8a8a,-11,-0.686802,2017-03-23 17:18:16,1.0,9,7
227326,Y,C_ID_8b04d4734e,69,N,1,B,879,M_ID_00a6ca8a8a,-10,-0.018122,2017-04-27 16:35:14,1.0,9,29
227367,Y,C_ID_8b04d4734e,69,N,1,B,879,M_ID_00a6ca8a8a,-9,-0.395588,2017-05-04 15:21:21,1.0,9,29
227523,Y,C_ID_8b04d4734e,69,N,1,B,879,M_ID_00a6ca8a8a,-9,-0.557574,2017-05-04 15:38:00,1.0,9,29
227637,Y,C_ID_8b04d4734e,69,N,2,C,879,M_ID_00a6ca8a8a,-9,0.455213,2017-05-30 13:58:39,1.0,9,29
227323,Y,C_ID_8b04d4734e,69,N,1,B,879,M_ID_00a6ca8a8a,-8,-0.706336,2017-06-20 11:48:07,1.0,9,29
227713,Y,C_ID_8b04d4734e,69,N,1,B,879,M_ID_00a6ca8a8a,-8,-0.706336,2017-06-21 11:42:05,1.0,9,29
227297,Y,C_ID_8b04d4734e,69,N,1,B,879,M_ID_00a6ca8a8a,-8,-0.706336,2017-06-29 11:45:07,1.0,9,29
227724,Y,C_ID_8b04d4734e,69,N,1,B,879,M_ID_00a6ca8a8a,-8,-0.644728,2017-06-29 13:59:42,1.0,9,29


In [3]:
plt.scatter(history.loc[history.card_id=='C_ID_8b04d4734e', 'installments'], history.loc[history.card_id=='C_ID_8b04d4734e', 'purchase_amount'])

NameError: name 'history' is not defined